In [412]:
import cvxpy as cp
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection  import train_test_split
import matplotlib.pyplot as plt

In [650]:
import numpy as np
from tqdm import tqdm
# from sklearn.metrics.pairwise import rbf_kernel
class SVMCust:
    def __init__(self) -> None:
        theta=1
        self.W=0
        self.b=0
        self.a=0
        self.C=0
        self.m=0
        self.f=0
    
    def kg(self,a, b, sigma=1):
        return np.exp((-(np.linalg.norm(a-b)**2))/(2*sigma**2))
        # print("dot:",np.dot(a,b))
        # # val=np.linalg.det(np.dot(a,b))
        # a=a.reshape((self.f,1))
        # b=b.reshape((self.f,1))
        # val=np.matmul(a.T,b)
        # print("as",a.shape, "b:",b.shape)
        # print(val.shape)
        return val
    # generating kernal matrix
    def k_mat(self,x, k_func=kg):
        m = x.shape[0]
        mat = np.zeros((m, m))
        for i in tqdm(range(m)):
            for j in range(m):
                mat[i, j] = self.kg(x[i], x[j])
        return mat

    def getAlphas(self):
        return self.a
    def getWeight(self):
        return self.W
    def getBias(self):
        return self.b
    def fit(self,X_train,y_train):
        print("shape:",X_train.shape)
        y_train[y_train==0]=-1
        (m,f)=X_train.shape
        self.f=f
        y_train=np.asarray(y_train)
        self.m=m
        X=np.array(X_train)
        # k = rbf_kernel(X)
        k=self.k_mat(X)
        # print(np.all(np.abs(k-k.T) < 1e-08))
        # print(k)
        G=np.zeros((m,m))
        print("shaope:",G.shape)
        print("m:",m)
        for i in tqdm(range(0,m)):
            # for j in range(0,i+1):
                # print("i:",i,"j:",j)
                # print("s:",(y[i]*(np.inner(X[i],np.transpose(X[j])))*y[j]).shape)
                G[i]=(y_train[i]*np.multiply(y_train,(k[i])))
                # G[j][i]=G[i][j]
        # G=y*k*np.transpose(y)
        # print(G)

        self.a = cp.Variable(m)
        self.C = cp.Parameter(nonneg=True)
        self.C.value = 0.02
        print("min eigne val:",np.linalg.eigvals(G).min())
        G = cp.atoms.affine.wraps.psd_wrap(G)
        obj = cp.Minimize( (0.5) * cp.quad_form(self.a,G)-np.ones((1,m))@ self.a)
        constraints = [0<=self.a,self.a<=self.C ,np.transpose(y_train)@self.a==0]
        prob = cp.Problem(obj, constraints)
        print("Is prob dcp:",prob.is_dcp())
        result = prob.solve()
        print(self.a.value)
        self.W=np.zeros(f,dtype='float64')
        tempa=np.array(self.a.value)

        print(tempa)
        for i in range(m):
            if tempa[i]>=0.0199:
                self.W+=y_train[i]* tempa[i]* X[i]
        cnt_pos_vec=0
        # print("W:",type(W))
        self.W=np.asarray(self.W,dtype='float64')
        for i in range(m):
            if tempa[i]>=0.0199:
                # print(X[i].dtype)
                
                # print("y:",self.W.dtype)
                # print("h",y.dtype)
                # print((self.W* np.transpose(X[i])).shape)
                self.b+=y_train[i]-np.matmul(self.W, np.transpose(X[i]))
                cnt_pos_vec+=1
        y_train[y_train==-1]=0
        self.b/=max(1,cnt_pos_vec)
    def predict(self,X):
        # ycap=np.matmul(self.W,np.transpose(X))
        matb=np.full((X.shape[0],1),self.b)
        matw=np.full((X.shape[0],X.shape[1]),self.W)
        # print(matb)
        # print(matw)
        # print("shapE,",np.matmul(X,np.transpose(self.W)).shape)
        print(np.matmul(X,np.transpose(self.W)))
        # print("fnal:,",np.add(matb,np.matmul(X,np.transpose(self.W))).shape)
        y_cap=self.b+np.transpose(np.matmul(X,np.transpose(self.W)))
        print(y_cap.shape)
        y_cap[y_cap<=0]=0
        y_cap[y_cap>0]=1
        return y_cap
        
        
        

        
        
        



    
    
    
    

In [651]:
import pandas as pd
import numpy as np
from sklearn import svm
import math

In [652]:
##global variables
tz=0
tpr_by_fpr_min_thresh=0

tc=0
target_column_name='income'

In [653]:
data=pd.read_csv("datasets/adult.csv")

In [654]:
data['income'].replace(['<=50K', '>50K'],
                        [0, 1], inplace=True)

In [655]:
list_of_catagorical_vars=['workclass','education',
       'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country',
]


In [656]:
from sklearn.preprocessing import LabelEncoder
encoded_to_orig_dict={}
for col_name in list_of_catagorical_vars:
    le = LabelEncoder()
    le.fit(data[col_name])
    data[col_name]=le.transform(data[col_name])
    

    encoded_to_orig_dict[col_name]= dict((v,k) for k,v in dict(zip(le.classes_, le.transform(le.classes_))).items())

In [657]:
traindt=data;
test=data[target_column_name]
traindt.drop(target_column_name,axis=1,inplace=True)


In [658]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(traindt,test, test_size=0.25) # 70% training and 30% test

In [659]:
clf=SVMCust()

In [660]:
# X_train.shape
# y_test=y_test[:500]
# X_test=X_test[:500]
# X_train=X_train[:1000]
# y_train=y_train[:1000]

In [661]:
from sklearn import preprocessing
x = X_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
X_train_norm= min_max_scaler.fit_transform(x)
X_test_norm=min_max_scaler.transform(X_test.values)
# df = pd.DataFrame(x_scaled)


In [662]:
pd.DataFrame( y_test).value_counts()

income
0         6209
1         1932
dtype: int64

In [663]:
clf.fit(X_train_norm[:5000], y_train[:5000])

shape: (5000, 14)


 33%|███▎      | 1638/5000 [01:12<03:11, 17.55it/s]

In [621]:
alphas=clf.getAlphas().value

In [636]:
alphas

array([0.  , 0.  , 0.  , ..., 0.02, 0.02, 0.  ])

In [638]:
alphas[alphas>=0.01999]=1
alphas[alphas<0.02]=0

In [639]:
alphas

array([0., 0., 0., ..., 1., 1., 0.])

In [640]:
pd.DataFrame(alphas).value_counts()

0.0    3861
1.0    1139
dtype: int64

In [641]:
y_train.value_counts()

0    18610
1     5810
Name: income, dtype: int64

In [642]:
y_pred=clf.predict(X_test_norm)

[0. 0. 0. ... 0. 0. 0.]
(8141,)


In [643]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.750522048888343


In [644]:
from sklearn.metrics import confusion_matrix

In [645]:
confusion_matrix(y_test,y_pred)

array([[6110,    0],
       [2031,    0]], dtype=int64)

In [646]:
test_df_final=pd.concat([X_test,y_test],axis=1)

In [647]:
all_test=pd.concat([X_train,y_train],axis=1)

In [648]:
all_test['Pred']=alphas

ValueError: Length of values (5000) does not match length of index (24420)

In [436]:
true_predictions=all_test.loc[all_test[target_column_name]==all_test['Pred']]
true_positive_predictions=true_predictions.loc[true_predictions['Pred']==1]
# true_positive_predictions
true_negative_predictons=true_predictions.loc[true_predictions['Pred']==0]

In [437]:
all_test[target_column_name].value_counts()

0    18514
1     5906
Name: income, dtype: int64

In [438]:
true_predictions[target_column_name].value_counts()

0    18514
1     5906
Name: income, dtype: int64

In [439]:
true_positive_predictions.drop('Pred',axis=1,inplace=True)
true_negative_predictons.drop('Pred',axis=1,inplace=True)

c:\Users\acer\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [440]:
true_predictions_x=true_predictions.drop([target_column_name,'Pred'],axis=1)
true_predictions_y=true_predictions[target_column_name]

In [441]:
true_predictions_x= true_predictions_x.loc[:, (true_predictions_x != true_predictions_x.iloc[0]).any()] 

In [442]:
list_of_features_with_non_0_var=list(true_predictions_x.columns)
list_of_features_with_non_0_var

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [443]:
true_negative_predictons.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

In [444]:
list_of_features_with_non_0_var

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [445]:
true_negative_predictons=true_negative_predictons[list_of_features_with_non_0_var+[target_column_name]]
true_positive_predictions=true_positive_predictions[list_of_features_with_non_0_var+[target_column_name]]

In [446]:

column_mean_for_STP_dict={}
column_mean_for_STN_dict={}
column_std_for_STP_dict={}
column_std_for_STN_dict={}
for column_name in list_of_features_with_non_0_var:
    if column_name not in list_of_catagorical_vars:
        print(column_name)
        column_mean_for_STP_dict[column_name]=true_positive_predictions[column_name].mean()
        column_mean_for_STN_dict[column_name]=true_negative_predictons[column_name].mean()
        column_std_for_STN_dict[column_name]=true_positive_predictions[column_name].std()
        column_std_for_STP_dict[column_name]=true_negative_predictons[column_name].std()

age
fnlwgt
education.num
capital.gain
capital.loss
hours.per.week


In [447]:
column_mean_for_STN_dict

{'age': 36.74813654531706,
 'fnlwgt': 190128.38813870584,
 'education.num': 9.580155557956141,
 'capital.gain': 149.1267149184401,
 'capital.loss': 54.011828886248246,
 'hours.per.week': 38.81365453170574}

In [448]:
column_mean_for_STP_dict

{'age': 44.099729089061974,
 'fnlwgt': 189128.2414493735,
 'education.num': 11.622925838130714,
 'capital.gain': 4149.221977649848,
 'capital.loss': 199.26413816457838,
 'hours.per.week': 45.48272942770064}

In [449]:
num_of_samples_in_STP=len(true_positive_predictions.columns)
num_of_samples_in_STN=len(true_negative_predictons.columns)

In [450]:
def getZmeanDiff(columnName):
    num=column_mean_for_STN_dict[column_name]+column_mean_for_STP_dict[column_name]
    dnum=((column_std_for_STN_dict[column_name]**2)/num_of_samples_in_STN)+((column_mean_for_STP_dict[column_name]**2)/num_of_samples_in_STP)
    dnum=math.sqrt(dnum)
    return num/dnum

In [451]:
from math import exp
from sklearn.utils import resample


def getChiSquaredTest(columnName):
    expected_dict={}
    obs_dict={}
    unique_val_in_stp=list(true_positive_predictions[column_name].unique())
    unique_val_in_stn=list(true_negative_predictons[col_name].unique())
    # print(unique_val_in_stp," ",unique_val_in_stn)
    obs_freq={}
    exp_freq={}
    for val in (unique_val_in_stn+ unique_val_in_stp) :
        obs_freq[val]=0
        exp_freq[val]=0
    # print(obs_freq)
    # print(exp_freq)
    tndict=pd.value_counts(true_negative_predictons[col_name]).to_dict()
    tpdict=pd.value_counts(true_positive_predictions[col_name]).to_dict()
    res=0
    # print("Res:",res)
    # print(obs_freq.keys())
    # print(exp_freq.keys())
    for val in (unique_val_in_stn+unique_val_in_stp):
        if val in tpdict.keys():
            obs_freq[val]=(tpdict[val]/ true_positive_predictions.shape[1])
        if val in tndict.keys():
            exp_freq[val]=(tndict[val]/ true_negative_predictons.shape[1])
        # print(obs_dic/t[val])
        # print(exp_)
        if exp_freq[val]==0:
            exp_freq[val]=0.2
        res+=((obs_freq[val]-exp_freq[val])**2)/exp_freq[val]
    return res
    
    
    


    

In [452]:
list_of_features_with_non_0_var

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [453]:
Z_diff_btw_two_means_dict={}
chi_squared_dict={}
for column_name in list_of_features_with_non_0_var:
    # print("cn:",column_name)
    if column_name not in list_of_catagorical_vars:
        Z_diff_btw_two_means_dict[column_name]=getZmeanDiff(column_name)
    else:
        chi_squared_dict[column_name]=getChiSquaredTest(column_name)

In [454]:
Z_diff_btw_two_means_dict

{'age': 6.906652428324939,
 'fnlwgt': 6.815417950008579,
 'education.num': 6.920176811301177,
 'capital.gain': 1.072349558212845,
 'capital.loss': 1.5481407792603785,
 'hours.per.week': 6.976513777007731}

In [455]:
chi_squared_dict

{'workclass': 597.1544492539992,
 'education': 606.199924455903,
 'marital.status': 597.3941317936818,
 'occupation': 608.6802274862059,
 'relationship': 594.5217913681499,
 'race': 593.2973015722316,
 'sex': 587.8984732937334,
 'native.country': 1155.4932428837628}

In [456]:
F=[]#
for column_name in list_of_features_with_non_0_var:
    if( column_name not in list_of_catagorical_vars and  Z_diff_btw_two_means_dict[column_name]>=tz):
        F+=[column_name]
    elif (column_name in list_of_catagorical_vars and chi_squared_dict[col_name]>=tc):
        F+=[column_name]

In [457]:
F

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [458]:
def get_tpr_to_fpr_ratio(df,column_name,thresh,target_column_name, iscategorical=0):
    tp=0
    fp=0
    fn=0
    tn=0
    if iscategorical==0:
        for index,row in df.iterrows():
            if(row[column_name]>=thresh):
                if(row[target_column_name]==1):
                    tp+=1
                else:
                    fp+=1
            else:
                if(row[target_column_name]==0):
                    tn+=1
                else:
                    fn+=1
    else:
        for index,row in df.iterrows():
            if(row[column_name]==thresh):
                if(row[target_column_name]==1):
                    tp+=1
                else:
                    fp+=1
            else:
                if(row[target_column_name]==0):
                    tn+=1
                else:
                    fn+=1

    tpr=tp/(fn+tp)
    fpr=fp/(fn+tp)
    if fpr==0:
        fpr=10000000
    # print("TPR:",tpr," Fpr:",fpr)
    return tpr/fpr

        
   
def get_aib_for_feature(column_name,df,step,target_column_name):
    myet=0
    aib=-1
    start_val=int(math.floor(df[column_name].min()))
    end_val=int(math.ceil(df[column_name].max()))
    step=max(1,int((end_val-start_val)/200))
    for poss_thres in range(start_val,end_val,step):
        val=get_tpr_to_fpr_ratio(df,column_name,poss_thres,target_column_name)
        # print("for thresh:",poss_thres," val:",val,"myet:", myet)
        if(myet<val):
            aib=poss_thres
            myet=val
    return aib





In [459]:
# def get_tpr_to_fpr_ratio(df,column_name,val,target_column_name):
def get_aib_for_cat_feature(column_name,df,target_column_name):
    local_dict={}
    for value in df[column_name].unique():
        local_dict[(column_name,value)]=get_tpr_to_fpr_ratio(df,column_name,value,target_column_name,iscategorical=1)
    return local_dict  
        
        


In [460]:
from tqdm import tqdm
# tempf=['Pregnancies']
aib_dict_for_F={}
aib_dict_for_cat_F={}
best_tpr_by_fpr_for_feature={}
for column_name in tqdm(F):
    if column_name not in list_of_catagorical_vars:
        aib_dict_for_F[column_name]=get_aib_for_feature(column_name,true_predictions,1,target_column_name)
        best_tpr_by_fpr_for_feature[column_name]=get_tpr_to_fpr_ratio(true_predictions,column_name,aib_dict_for_F[column_name],target_column_name)
    else :
        temp_dict=get_aib_for_cat_feature(column_name,true_predictions,target_column_name)
        aib_dict_for_cat_F.update(temp_dict)
        best_tpr_by_fpr_for_feature.update(temp_dict)
        

100%|██████████| 14/14 [15:49<00:00, 67.82s/it] 


In [461]:
ranked_best_tpr_by_fpr_features=sorted(best_tpr_by_fpr_for_feature, key=lambda k: best_tpr_by_fpr_for_feature[k],reverse=True)

In [462]:
all_rule_list=[]
for value in ranked_best_tpr_by_fpr_features:
    
    if type(value) is not tuple:
        temp_dict={
            "column_name":value,
            "sign":">",
            "thresh":aib_dict_for_F[value]
        }
    else :
        temp_dict={
            "column_name":value[0],
            "sign":"==",
            "thresh":value[1]
        }

    all_rule_list.append(temp_dict)
all_rule_list

[{'column_name': 'capital.gain', 'sign': '>', 'thresh': 34431},
 {'column_name': 'capital.loss', 'sign': '>', 'thresh': 2394},
 {'column_name': 'education', 'sign': '==', 'thresh': 10},
 {'column_name': 'education.num', 'sign': '>', 'thresh': 15},
 {'column_name': 'education', 'sign': '==', 'thresh': 14},
 {'column_name': 'education', 'sign': '==', 'thresh': 12},
 {'column_name': 'workclass', 'sign': '==', 'thresh': 5},
 {'column_name': 'occupation', 'sign': '==', 'thresh': 4},
 {'column_name': 'relationship', 'sign': '==', 'thresh': 5},
 {'column_name': 'native.country', 'sign': '==', 'thresh': 10},
 {'column_name': 'occupation', 'sign': '==', 'thresh': 10},
 {'column_name': 'relationship', 'sign': '==', 'thresh': 0},
 {'column_name': 'marital.status', 'sign': '==', 'thresh': 2},
 {'column_name': 'native.country', 'sign': '==', 'thresh': 24},
 {'column_name': 'hours.per.week', 'sign': '>', 'thresh': 50},
 {'column_name': 'education', 'sign': '==', 'thresh': 9},
 {'column_name': 'nativ

In [463]:
def isRuleSatisfied(rule_list,row):
    for rule in rule_list:
        cname=rule['column_name']
        if rule['sign']=='>':
            if row[cname]<=rule['thresh']:
                return False
        elif rule['sign']=='==':
            if row[cname]!=rule['thresh']:
                return False
    return True

In [464]:
def get_tpr_by_fpr_for_rule_list(rule_list,df,target_column_name):
    tp=0
    tn=0
    fp=0
    fn=0
    for index,row in df.iterrows():
        
        if(isRuleSatisfied(rule_list,row)):
            if(row[target_column_name]==1):
                tp+=1
            else:
                fp+=1
        else:
            if(row[target_column_name]==0):
                tn+=1
            else:
                fn+=1
    tpr=0
    fpr=0
    if tp+fp==0:
        tpr=tp/0.1
    else:
        tpr=tp/(fn+tp)
    if fn+tp==0:
        fpr=fp/0.1
    else:
        fpr=fp/(fn+tp)
    # print("tp:",tp," fp:",fp," tn:",tn," fn:",fn,"TPR:",tpr," Fpr:",fpr)
    if fpr==0:
        return (tpr/2000000,0)
   
    return (tpr/fpr,fpr)
    
    
    

In [465]:
final_rule_list=[]

In [466]:
# def add_rule_rec(df,all_rule_list,pres_ind,pres_rule_list,last_tpr_by_fpr_ratio,target_column_name):
def delete_rows_satisfying_rule(df,rule_list,target_column_name):
    list_of_ind_to_drop=[]
    ind=0
    for index,row in df.iterrows():
        # print("ind:",index)
    
        if(isRuleSatisfied(rule_list,row)):
            if(row[target_column_name]==1):
                list_of_ind_to_drop+=[ind]
        else:
            if(row[target_column_name]==0):
                list_of_ind_to_drop+=[ind]
        ind+=1
    df.drop(df.index[[list_of_ind_to_drop]],inplace=True)
    return df
    



In [467]:
result_rule_list=[]
df=true_predictions.copy()
for init_rule_ind in tqdm(range(0,len(all_rule_list))):
    init_rule_list=[all_rule_list[init_rule_ind]]
    (init_tpr_by_fpr,_)=get_tpr_by_fpr_for_rule_list(init_rule_list,df,target_column_name)
    # print("rule list:", init_rule_list)
    # print("tpr/fppr:",init_tpr_by_fpr)
    # print("df shape::",df.shape)
    new_rule_list=init_rule_list.copy()
    last_ratio=tpr_by_fpr_min_thresh
    vis=[0]*len(all_rule_list)
    vis[init_rule_ind]=1
    max_tpr_fpr_ratio=init_tpr_by_fpr
    max_tpr_fpr_ratio_ind=-1
    while get_tpr_by_fpr_for_rule_list(new_rule_list,df,target_column_name)[0]> last_ratio:
        init_rule_list=new_rule_list
        max_tpr_fpr_ratio=0
        max_tpr_fpr_ratio_ind=-1
        for new_rule_ind in range(len(all_rule_list)):
            if(vis[new_rule_ind]==1):
                continue
            temp_new_rule_list=new_rule_list.copy()
            temp_new_rule_list+=[all_rule_list[new_rule_ind]]
            (new_tpr_by_fpr,fpr_for_list)=get_tpr_by_fpr_for_rule_list(temp_new_rule_list,df,target_column_name)
            # print("for ind:",new_rule_ind, "new rat:",new_tpr_by_fpr," fpr_for_list:",fpr_for_list)
            if new_tpr_by_fpr>max_tpr_fpr_ratio and fpr_for_list!=0:
                max_tpr_fpr_ratio=new_tpr_by_fpr
                max_tpr_fpr_ratio_ind=new_rule_ind
        # print("new tpr/fpr",max_tpr_fpr_ratio)
        if max_tpr_fpr_ratio_ind==-1:
            break
        # if max_tpr_fpr_ratio>tpr_by_fpr_min_thresh:
        
        vis[max_tpr_fpr_ratio_ind]=1
        new_rule_list+=[all_rule_list[max_tpr_fpr_ratio_ind]]
        
    # print( "init rule list:",init_rule_list)
    result_rule_list.append(init_rule_list)
    #removing rows from df which are already classified
    # if max_tpr_fpr_ratio_ind==-1:
    #     continue
    df=delete_rows_satisfying_rule(df,new_rule_list,target_column_name)
    # print("df shape:",df.shape)
            
        



    



  0%|          | 0/108 [00:00<?, ?it/s]c:\Users\acer\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexes\base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
100%|██████████| 108/108 [17:07<00:00,  9.51s/it]  


In [468]:
result_rule_list

[[{'column_name': 'capital.gain', 'sign': '>', 'thresh': 34431},
  {'column_name': 'native.country', 'sign': '==', 'thresh': 39},
  {'column_name': 'race', 'sign': '==', 'thresh': 4},
  {'column_name': 'sex', 'sign': '==', 'thresh': 1},
  {'column_name': 'relationship', 'sign': '==', 'thresh': 0},
  {'column_name': 'marital.status', 'sign': '==', 'thresh': 2},
  {'column_name': 'age', 'sign': '>', 'thresh': 42},
  {'column_name': 'workclass', 'sign': '==', 'thresh': 6}],
 [{'column_name': 'capital.loss', 'sign': '>', 'thresh': 2394}],
 [{'column_name': 'education', 'sign': '==', 'thresh': 10}],
 [{'column_name': 'education.num', 'sign': '>', 'thresh': 15}],
 [{'column_name': 'education', 'sign': '==', 'thresh': 14}],
 [{'column_name': 'education', 'sign': '==', 'thresh': 12}],
 [{'column_name': 'workclass', 'sign': '==', 'thresh': 5}],
 [{'column_name': 'occupation', 'sign': '==', 'thresh': 4}],
 [{'column_name': 'relationship', 'sign': '==', 'thresh': 5}],
 [{'column_name': 'native.co

In [469]:
def get_rule_list_predictions(rule_list,df):
    y_pred=[]
    for index,row in df.iterrows():
        for rule in rule_list:
            if isRuleSatisfied(rule,row):
                y_pred+=[1]
                break
        else:
            y_pred+=[0]
           
    return y_pred

                
    

In [470]:
from sklearn import metrics

In [471]:
true_predictions[target_column_name].value_counts()

0    18514
1     5906
Name: income, dtype: int64

In [472]:
post_pruned_rule_list=[]
last_auc_area=0
for rule in tqdm(result_rule_list):
    temp_y_pred=get_rule_list_predictions(post_pruned_rule_list+[rule],true_predictions)
    # print(temp_y_pred)
    # print(true_predictions)
    # print(" pred len:",len(temp_y_pred))
    fpr, tpr, thresholds = metrics.roc_curve(true_predictions[target_column_name],temp_y_pred)
    # pres_auc=metrics.auc(fpr,tpr)
    # print("pres auc",pres_auc)
    if(pres_auc-last_auc_area>=0.05):
        post_pruned_rule_list.append(rule)
        last_auc_area=pres_auc

100%|██████████| 108/108 [02:06<00:00,  1.17s/it]


In [473]:
post_pruned_rule_list

[[{'column_name': 'capital.gain', 'sign': '>', 'thresh': 34431},
  {'column_name': 'native.country', 'sign': '==', 'thresh': 39},
  {'column_name': 'race', 'sign': '==', 'thresh': 4},
  {'column_name': 'sex', 'sign': '==', 'thresh': 1},
  {'column_name': 'relationship', 'sign': '==', 'thresh': 0},
  {'column_name': 'marital.status', 'sign': '==', 'thresh': 2},
  {'column_name': 'age', 'sign': '>', 'thresh': 42},
  {'column_name': 'workclass', 'sign': '==', 'thresh': 6}]]

In [474]:
def check_accuracy_of_rules_for_dataframe(rule_list,df,target_column_name):
    fp=0
    fn=0
    tp=0
    tn=0
    for index,row in df.iterrows():
        for rule in rule_list:
            if isRuleSatisfied(rule,row):
                if row[target_column_name]==1:
                    tp+=1
                else:
                    fp+=1
                break
        else:
            if row[target_column_name]==0:
                tn+=1
            else:
                fn+=1
    return (tp,tn,fp,fn)

In [475]:
(tp,tn,fp,fn)=check_accuracy_of_rules_for_dataframe(post_pruned_rule_list,test_df_final,target_column_name)

In [476]:
print("Accuracy of the rules:",(tp+tn)/(tp+tn+fp+fn))

Accuracy of the rules: 0.7623142120132662


In [477]:
def print_rule_list(rule_list):
    final_res="\n"
    for rules in rule_list:
        res="IF "
        # print("r:",rules[0])
        for i in range(0,len(rules)):
            if i>0:
                res+=" AND "
            res+=rules[i]['column_name']
            res+=" "
            res+=rules[i]['sign']
            res+=" "
            if rules[i]['column_name'] in list_of_catagorical_vars:
                val=rules[i]['thresh']
                cname=rules[i]['column_name']    
                res+=str(encoded_to_orig_dict[cname][val])
            else:
                res+=str(rules[i]['thresh'])
        res+=" THEN OUTCOME IS 1"
        # print("res:",res)
        final_res+="\n"
        final_res+=res
    print(final_res)        

In [478]:
print_rule_list(post_pruned_rule_list)




IF capital.gain > 34431 AND native.country == United-States AND race == White AND sex == Male AND relationship == Husband AND marital.status == Married-civ-spouse AND age > 42 AND workclass == Self-emp-not-inc THEN OUTCOME IS 1
